### Highways England API:

https://webtris.highwaysengland.co.uk/api/swagger/ui/index#!/Sites/Sites_Index_0

In [ ]:
!pip install requests

In [5]:
import requests
import json

#### Simple function to assemble API get requests for Highways API

In [6]:
def build_url(
    api_url = 'http://webtris.highwaysengland.co.uk/api/v1.0/',
    endpoint = 'reports/daily',
    query_params = {}
):
    query_params_str = '&'.join([f'{k}={v}' for k, v in query_params.items()])

    request_url = f'{api_url}/{endpoint}?{query_params_str}'

    return request_url

In [7]:
def get_request(url):

    response = requests.get(url)
    
    response_json = json.loads(response.text)

    return response_json

#### Get site data

In [8]:
query_params = {}

request_url = build_url(
    endpoint='sites/8188',
    query_params=query_params
)

response_json = get_request(request_url)

print(json.dumps(response_json, indent=4))


filename = "site_8188.json"

with open(filename, 'w') as f:
    
    json.dump(response_json, f)
    
    print(f'written to file: {filename}')

{
    "row_count": 1,
    "sites": [
        {
            "Id": "8188",
            "Name": "TMU Site 7001/1 on link A46 northbound between B439 and A435 near Alcester; GPS Ref: 407983;251953; Northbound",
            "Description": "7001/1",
            "Longitude": -1.88469556975806,
            "Latitude": 52.1658012090139,
            "Status": "Active"
        }
    ]
}
written to file: site_8188.json


#### Get report data

In [ ]:
query_params = {
    'sites': 8188,
    'start_date': 31032016,
    'end_date': 31032017,
    'page': 1,
    'page_size': 50
}

request_url = build_url(query_params=query_params)

response_json = get_request(request_url)

print(json.dumps(response_json, indent=4))

#### API responses include links to previous and next pages of data, these can be used to automate pagination through them all

In [ ]:
response_json['Header']['links'][0]['href']

In [ ]:
def get_next_url(response):
    
    header = response['Header']
    
    links = header['links']
    
    for link in links:
        
        if link['rel'] == 'nextPage':
            
            return link['href']
    
    return None

In [ ]:
def get_all_pages(api_url):
    
    data = []
    
    next_url = api_url
    
    while next_url:
        
        print(next_url)
        
        response_json = get_request(next_url)

        next_url = get_next_url(response_json)

        data += response_json['Rows']
    
    return data

In [ ]:
params = {
    'sites': 8188,
    'start_date': 31012020,
    'end_date':   31072020,
    'page': 1,
    'page_size': 50
}

request_url = build_url(query_params=params)

all_pages = get_all_pages(request_url)

# print(json.dumps(all_pages, indent=4))



filename = f"report_daily_{params['sites']}_{params['start_date']}_{params['end_date']}.json"

with open(filename, 'w') as f:
    
    json.dump(all_pages, f)
    
    print(f'written to file: {filename}')